导入所需最少依赖。

# Search Agent

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re
import requests  # 新增用于真实搜索

LOCAL_MODEL_PATH = r"C:/Users/nashk/Documents/nashknight/search_agent/Qwen3-0.6B"

c:\Users\nashk\.conda\envs\pytorch-gpu\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


定义并加载最小模型+分词器函数。

In [4]:
def load():
    tok = AutoTokenizer.from_pretrained(LOCAL_MODEL_PATH, local_files_only=True)
    m = AutoModelForCausalLM.from_pretrained(LOCAL_MODEL_PATH, local_files_only=True, device_map="auto" if torch.cuda.is_available() else None)
    return m, tok

model, tokenizer = load()

定义最小生成函数，返回原始与处理后输出。

In [98]:
def generate(prompt, max_new_tokens=512):
    msgs = [{"role":"user","content":prompt}]
    if hasattr(tokenizer, "apply_chat_template"):
        text = tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
    else:
        text = prompt
    inp = tokenizer(text, return_tensors="pt")
    for k,v in inp.items(): inp[k] = v.to(model.device)
    ids = model.generate(**inp, max_new_tokens=max_new_tokens)
    gen_ids = ids[0][inp['input_ids'].shape[-1]:]  # 仅保留新生成部分
    raw_text = tokenizer.decode(gen_ids, skip_special_tokens=False)
    clean_text = tokenizer.decode(gen_ids, skip_special_tokens=True)
    return gen_ids.tolist(), raw_text, clean_text


def clear_model_cache():
    """清空模型的KV缓存,让模型忘记之前的上下文"""
    if hasattr(model, 'past_key_values'):
        model.past_key_values = None
    # 清空CUDA缓存(如果使用GPU)
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

执行一次简单测试并输出原始与处理后结果，确保模型导入和基本功能正常。

In [6]:
tokens, raw_text, clean_text = generate("今天我心情很好")
print("RAW_ALL_TOKENS_TEXT:\n", raw_text)
print("\n\nCLEAN_RESPONSE:\n", clean_text)

RAW_ALL_TOKENS_TEXT:
 <think>
好的，用户说今天心情很好。首先，我需要确认用户是否在寻求情感支持或只是表达心情。用户可能希望得到一些鼓励或陪伴，但也要确保回复友好且不显得刻意。

接下来，我应该考虑用户可能的深层需求。也许他们需要一些安慰，或者想分享自己的感受。这时候，回应应该既肯定他们的积极情绪，又提供一些积极的建议或兴趣点，比如一起做喜欢的事情，或者推荐一些放松的活动。

同时，要注意语气要温暖，避免使用过于正式或机械的表达。用户可能希望得到真诚的回应，而不是简单的肯定。所以，我需要用一些具体的例子，比如一起看电影、散步，或者做一些放松的活动，让对方感受到被理解和关心。

另外，还要考虑用户是否有其他潜在的问题，比如是否需要更多的支持或建议。这时候，可以询问他们是否需要进一步的帮助，或者是否愿意分享更多关于他们的心情。

最后，确保回复简洁明了，避免冗长，同时保持自然和真诚。这样用户会感受到被重视，同时也愿意继续交流。
</think>

哇！今天过得真开心呀！要不要一起分享一下呢？或者我们可以做点让自己放松的事情，比如看电影、散步，或者聊聊天？你有想做什么吗？😊<|im_end|>


CLEAN_RESPONSE:
 <think>
好的，用户说今天心情很好。首先，我需要确认用户是否在寻求情感支持或只是表达心情。用户可能希望得到一些鼓励或陪伴，但也要确保回复友好且不显得刻意。

接下来，我应该考虑用户可能的深层需求。也许他们需要一些安慰，或者想分享自己的感受。这时候，回应应该既肯定他们的积极情绪，又提供一些积极的建议或兴趣点，比如一起做喜欢的事情，或者推荐一些放松的活动。

同时，要注意语气要温暖，避免使用过于正式或机械的表达。用户可能希望得到真诚的回应，而不是简单的肯定。所以，我需要用一些具体的例子，比如一起看电影、散步，或者做一些放松的活动，让对方感受到被理解和关心。

另外，还要考虑用户是否有其他潜在的问题，比如是否需要更多的支持或建议。这时候，可以询问他们是否需要进一步的帮助，或者是否愿意分享更多关于他们的心情。

最后，确保回复简洁明了，避免冗长，同时保持自然和真诚。这样用户会感受到被重视，同时也愿意继续交流。
</think>

哇！今天过得真开心呀！要不要一起分享一下呢？或者我们可以做点让自己放松的事情，比如看电影、

In [138]:
# Search Agent v2.0 core (真实搜索集成 + 多轮引用追踪)

# 长度控制参数
MAX_SOURCES_PER_SEARCH = 5  # 每次搜索返回的最大信息条数
MAX_SOURCE_DESC_LEN = 400    # 单条信息description的最大长度
MAX_FORMATTED_SOURCES_LEN = 1500  # 格式化搜索结果在prompt中的最大长度
MAX_RAW_DISPLAY_LEN = 1000   # 显示raw_responses的最大长度

BASE_PROMPT = (
    "你是一个智能搜索助手。你的任务是分析用户问题,判断是否需要搜索来获取信息。\n\n"
    "**核心判断原则:**\n"
    "在分析问题时,如果遇到以下情况,必须使用<search>标签搜索:\n"
    "1. 需要实时数据(股价、天气、新闻、汇率等会变化的信息)\n"
    "2. 需要最新信息(当前状态、今天/现在的情况)\n"
    "3. 需要具体事实(某个公司的数据、某个地点的情况、某个产品的参数)\n"
    "4. 你不确定答案,或答案可能过时\n"
    "5. 用户明确要求查询、搜索、查找信息\n\n"
    "如果是以下情况,可以直接回答:\n"
    "1. 常识性问题(如何做某事、概念解释)\n"
    "2. 主观问题(建议、意见)\n"
    "3. 数学计算、逻辑推理\n\n"
    "**确定需要搜索时的输出格式**:\n"
    "<search>简短查询词</search>\n\n"
    "**不需要搜索时,直接给出答案。**\n\n"
    "**关键规则:**\n"
    "- <search>标签单独成行,内容3-10字\n"
    "- 没搜索前不要编造数字、日期等事实\n"
    "- 宁可多搜索,不要猜测\n"
    "- 如果已有足够信息,直接回答并列出参考来源,不要再输出<search>\n"
)

# Jina Search API 配置
JINA_API_KEY = "jina_800f62ec9cc745e09f058c4652a961feziG6FeCa71toa9my7gXm3prQbJaF"
JINA_SEARCH_ENDPOINT = "https://s.jina.ai"
PROXIES = {"http": "http://127.0.0.1:7890", "https": "http://127.0.0.1:7890"}


def fetch_search_result(query: str, use_proxy: bool = True, max_sources: int = MAX_SOURCES_PER_SEARCH) -> dict:
    """调用 Jina Search 获取结果并返回结构化字典。失败时返回错误信息。"""
    headers = {
        "Accept": "application/json",
        "Authorization": f"Bearer {JINA_API_KEY}",
        "X-Respond-With": "no-content"
    }
    url = f"{JINA_SEARCH_ENDPOINT}/?q={requests.utils.quote(query)}"
    proxies_cfg = PROXIES if use_proxy else None
    try:
        resp = requests.get(url, headers=headers, proxies=proxies_cfg, timeout=12)
        resp.raise_for_status()
        results_json = resp.json()
        data = results_json.get('data', [])
        if not data:
            return {"error": f"未找到结果: {query}", "sources": {}}
        sources = {}
        for i, item in enumerate(data[:max_sources], 1):
            title = (item.get('title') or '').strip()[:120]
            description = (item.get('description') or '').strip()[:MAX_SOURCE_DESC_LEN]
            url_link = (item.get('url') or '').strip()
            if description and url_link:
                sources[f"信息{i}"] = {"url": url_link, "description": description, "title": title}
        return {"sources": sources, "error": None} if sources else {"error": "结果无有效内容", "sources": {}}
    except Exception as e:
        return {"error": f"{type(e).__name__}: {e}", "sources": {}}


def extract_search_query(raw: str):
    """提取<search>标签,支持单独成行或内嵌格式"""
    match = re.search(r'<search>\s*([^<\n]+?)\s*</search>', raw, re.IGNORECASE)
    if match:
        query = match.group(1).strip()
        if query and len(query) <= 80:
            return query, raw[:match.end()]
    return None, None


def clean_final_response(text: str) -> str:
    """清理最终回答,移除 <think> 和 <search> 标签及其内容,只保留用户可见部分。"""
    text = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r'<search>.*?</search>', '', text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r'</?(?:think|search)>', '', text, re.IGNORECASE)
    text = '\n'.join(line.strip() for line in text.split('\n') if line.strip())
    return text.strip()


def format_sources_for_prompt(sources_dict: dict, used_sources: dict) -> str:
    """格式化信息字典为 prompt 字符串,并追踪已使用的来源。"""
    if not sources_dict:
        return "[无可用信息]"
    lines = ["**搜索结果:**"]
    for key, val in sources_dict.items():
        desc = val.get('description', '')[:300]
        url = val.get('url', '')
        title = val.get('title', '未命名')
        lines.append(f"{key}({title}): {desc}")
        lines.append(f"  URL: {url}")
        if url and url not in used_sources:
            used_sources[url] = title
    return "\n".join(lines)


def run_search_agent(user_query, max_rounds=5, max_new_tokens=512, use_proxy=True):
    # 清空模型缓存,确保每个新查询都是独立的上下文
    clear_model_cache()
    
    rounds = []
    prompt = f"{BASE_PROMPT}\n用户问题: {user_query}"
    search_count = 0
    used_sources = {}
    
    for r in range(1, max_rounds + 1):
        _, raw, clean = generate(prompt, max_new_tokens=max_new_tokens)
        entry = {"round": r, "raw": raw, "clean": clean, "prompt": prompt}  # 保存完整prompt
        
        q, raw_trunc = extract_search_query(raw)
        if q:
            search_count += 1
            entry['raw'] = raw_trunc if raw_trunc else raw
            entry['search'] = q
            
            result_dict = fetch_search_result(q, use_proxy=use_proxy)
            entry['search_result'] = result_dict
            
            if result_dict.get('error'):
                print(f"⚠️ 搜索失败: {result_dict['error']}")
                rounds.append(entry)
                break
            
            sources = result_dict.get('sources', {})
            formatted_sources = format_sources_for_prompt(sources, used_sources)
            entry['formatted_sources'] = formatted_sources[:MAX_FORMATTED_SOURCES_LEN]
            rounds.append(entry)
            
            # 构造下一轮 prompt - 完整详细地说明任务和当前状态
            prompt = (
                f"{BASE_PROMPT}\n\n"
                f"=== 任务回顾 ===\n"
                f"用户的原始问题(query)是: {user_query}\n\n"
                f"你是一个智能搜索助手,负责为用户查询信息并给出专业答案。\n"
                f"你在第{search_count}轮搜索中已经查询了关键词'{q}',获得了以下搜索结果:\n\n"
                f"{formatted_sources[:MAX_FORMATTED_SOURCES_LEN]}\n\n"
                f"=== 当前任务(极其重要!) ===\n"
                f"现在你需要分析这些搜索结果,判断是否足够回答用户的问题:\n\n"
                f"**情况1: 信息已经足够 (这是最核心的环节!)**\n"
                f"如果以上搜索结果包含了足够的信息来回答'{user_query}',请务必按以下步骤操作:\n\n"
                f"第一步(最重要!必须执行!): 直接回答用户问题(query)\n"
                f"  - 用1-3句话给出核心答案,包含具体数字/事实，回答要围绕query，简明扼要\n"
                f"  - 例如用户问股价,你要说'特斯拉(TSLA)当前股价为XXX美元'\n"
                f"  - 严禁照抄输出示例！一定要根据先前的所有信息和分析自己总结！\n"
                f"  - 不要跳过这一步!这是用户最需要的!\n\n"
                f"第二步: 补充详细说明(可选)\n"
                f"  - 如果有额外有价值的信息,简要补充\n"
                f"  - 如涨跌幅、市值等相关数据\n\n"
                f"第三步: 列出参考来源\n"
                f"  - 格式:\n"
                f"    参考来源:\n"
                f"    [1] URL1\n"
                f"    [2] URL2\n\n"
                f"第四步: 不要再输出<search>标签\n\n"
                f"输出示例:\n"
                f"特斯拉(TSLA)当前股价为XXX美元,较前一交易日上涨XX%。\n\n"
                f"参考来源:\n"
                f"[1] https://...\n"
                f"[2] https://...\n\n"
                f"**情况2: 信息不足,需要继续搜索**\n"
                f"如果搜索结果不够详细,或者缺少关键信息:\n"
                f"1. 分析缺少什么信息\n"
                f"2. 输出<search>新的查询词</search>来获取更多细节\n"
                f"3. 注意不要重复搜索相同的关键词\n\n"
                f"=== 关键提醒 ===\n"
                f"- 这些搜索结果都是你自己查询得到的,不是用户提供的\n"
                f"- 你必须给出实质性答案,不能只列出链接!用户需要你的总结!\n"
                f"- 信息足够就立即回答,先答案后链接,不要只有链接没有答案!\n"
                f"- 回答要专业简洁,直接针对问题,不要说'我需要分析'这类话\n\n"
                f"现在请立即给出你的答案(记住:先回答问题,再列参考来源):")
            continue
        
        # 无 <search> 标签,认为是最终答案
        entry['clean'] = clean_final_response(clean)
        entry['used_sources'] = used_sources.copy()
        rounds.append(entry)
        break
    
    return rounds, used_sources


def show_rounds(rounds, used_sources=None, user_query=None):
    if user_query:
        print(f"Query: {user_query}")
    
    for i, info in enumerate(rounds, 1):
        print(f"\n{'='*80}")
        print(f"=== Round {info['round']} ===")
        
        print('Raw_responses:')
        print(info['raw'][:MAX_RAW_DISPLAY_LEN] + '...' if len(info['raw']) > MAX_RAW_DISPLAY_LEN else info['raw'])
        
        if 'search' in info:
            print(f"\n🔍 search_content: {info['search']}")
        
        if 'search_result' in info:
            result = info['search_result']
            if result.get('error'):
                print(f"\n⚠️ 搜索错误: {result['error']}")
            else:
                sources = result.get('sources', {})
                print(f"\n📚 搜索到 {len(sources)} 条信息（仅展示前3条）:")
                for key, val in list(sources.items())[:3]:
                    print(f"  {key}: {val.get('description', '')[:100]}...")
        
        print()
        is_last = (i == len(rounds))
        if is_last:
            print('✅ Clean_responses (最终答案):')
            print(info['clean'] if info['clean'] else '[无最终回答]')
            if used_sources:
                print(f"\n📎 本次对话所有追踪的链接 (共{len(used_sources)}个):")
                for idx, (url, title) in enumerate(list(used_sources.items())[:], 1):
                    print(f"  [{idx}] {title}\n      {url}")
        print('='*80)


In [107]:
# Tests for Search Agent v2.0
print("\n### Test 1: 心情真好 ###")
user_query = "今天我心情真好"
rounds1, sources1 = run_search_agent(user_query)
show_rounds(rounds1, sources1, user_query)



### Test 1: 心情真好 ###
Query: 今天我心情真好

=== Round 1 ===
Raw_responses:
<think>
好的，用户说“今天我心情真好”，看起来他们是在表达今天的积极情绪。首先，我需要判断是否需要搜索来获取信息。根据核心判断原则，如果问题需要实时数据或最新信息，或者不确定答案，或者用户明确要求查询，就需要搜索。

用户的问题只是表达心情，没有提到需要具体的数据，比如天气、新闻、汇率等。所以这里应该不需要搜索。用户可能只是想确认今天的感受，或者需要进一步的信息来支持这种感受。但根据规则，如果问题不涉及需要搜索的情况，直接回答即可。

因此，正确的做法是直接给出回答，不需要使用<search>标签。确认没有其他特殊情况，比如需要实时数据时，但这里没有相关数据，所以直接回答。
</think>

今天心情真好，可以分享一下最近的动态哦~<|im_end|>

✅ Clean_responses (最终答案):
今天心情真好，可以分享一下最近的动态哦~
Query: 今天我心情真好

=== Round 1 ===
Raw_responses:
<think>
好的，用户说“今天我心情真好”，看起来他们是在表达今天的积极情绪。首先，我需要判断是否需要搜索来获取信息。根据核心判断原则，如果问题需要实时数据或最新信息，或者不确定答案，或者用户明确要求查询，就需要搜索。

用户的问题只是表达心情，没有提到需要具体的数据，比如天气、新闻、汇率等。所以这里应该不需要搜索。用户可能只是想确认今天的感受，或者需要进一步的信息来支持这种感受。但根据规则，如果问题不涉及需要搜索的情况，直接回答即可。

因此，正确的做法是直接给出回答，不需要使用<search>标签。确认没有其他特殊情况，比如需要实时数据时，但这里没有相关数据，所以直接回答。
</think>

今天心情真好，可以分享一下最近的动态哦~<|im_end|>

✅ Clean_responses (最终答案):
今天心情真好，可以分享一下最近的动态哦~


In [111]:
print("\n### Test 2: 特斯拉股价 ###")
user_query = "查询特斯拉的实时股价"
rounds2, sources2 = run_search_agent(user_query, max_rounds=5)
show_rounds(rounds2, sources2, user_query)



### Test 2: 特斯拉股价 ###
Query: 查询特斯拉的实时股价

=== Round 1 ===
Raw_responses:
<think>
好的，用户的问题是“查询特斯拉的实时股价”。我需要先判断是否需要搜索。根据核心判断原则，如果需要实时数据，比如股价、天气、新闻、汇率等，应该使用<search>标签。特斯拉的股价确实实时更新，所以应该搜索相关的信息。接下来要生成一个简短的搜索词，比如“特斯拉实时股价查询”。然后确认没有编造事实，直接给出回答。不需要多搜索，因为信息已经足够。所以最终输出应该是<search>特斯拉实时股价查询</search>

🔍 search_content: 特斯拉实时股价查询

📚 搜索到 5 条信息（仅展示前3条）:
  信息1: 特斯拉(TSLA) ; 平均成交量(3个月). 89.78M ; 1年涨跌幅. 10.93% ; 每股账面价值. 24.05 ; 公允价值. unlock 解锁 ; 按公允价值计的上涨潜力. unlo...
  信息2: 今开: 402.320. 最高 ; 昨收: 395.230. 最低 ; 换手率: 3.02%. 市盈TTM ; 振幅: 4.82%. 市盈率静 ; 量比: 1.20. 市净率 ......
  信息3: 前收市價 391.09 · 開市 402.16 · 買入價 416.90 x 200 · 賣出價 417.39 x 100 · 今日波幅 401.09 - 421.71 · 52 週波幅 214.25...


=== Round 2 ===
Raw_responses:
<think>
好的，用户的问题是查询特斯拉的实时股价。我需要先看看提供的搜索结果有没有足够信息来回答这个问题。

首先看搜索结果，信息1提到了特斯拉(TSLA)的平均成交量、1年涨跌幅、每股账面价值、公允价值和上涨潜力。这些数据都是实时的，所以可能可以回答股价。信息2和信息3也提供了股价、成交量、涨跌幅等数据，URL链接显示来源是不同的网站，但内容中已经包含实时数据。信息4和5虽然提到了财务数据，但可能不是实时的。综合来看，所有结果都提到了实时股价，所以可以确定信息足够。

接下来要按照步骤回答用户，直接给出股价数字，补充相关数据，并列出参考来源。不需要使用<search>标签，因为信息已经足够。确

In [140]:
print("\n### Test 3: 奥运会奖牌榜 ###")
user_query = "请告诉我2024年夏季奥运会的最新奖牌榜排名情况"
rounds3, sources3 = run_search_agent(user_query, max_rounds=7)
show_rounds(rounds3, sources3, user_query)


### Test 3: 奥运会奖牌榜 ###
Query: 请告诉我2024年夏季奥运会的最新奖牌榜排名情况

=== Round 1 ===
Raw_responses:
<think>
好的，用户问的是2024年夏季奥运会的最新奖牌榜排名情况。首先，我需要确定是否需要搜索。根据核心判断原则，如果需要实时数据或最新信息，应该使用<search>标签。奖牌榜的排名通常会变化，所以可能需要实时数据。用户可能想知道最新的数据，所以应该搜索相关的信息。需要检查是否有足够的信息，但如果没有，就用搜索。所以正确的做法是输出<search>2024年夏季奥运会奖牌榜最新排名</search>

🔍 search_content: 2024年夏季奥运会奖牌榜最新排名

📚 搜索到 5 条信息（仅展示前3条）:
  信息1: 2024年巴黎奥运会 奖牌榜 ; 阿尔巴尼亚. ALB. 阿尔巴尼亚 ; 阿尔及利亚. ALG. 阿尔及利亚 ; 阿根廷. ARG. 阿根廷 ; 亚美尼亚. ARM. 亚美尼亚 ; 澳大利亚. AUS...
  信息2: 本届奥运会美国代表团以40金44银42铜126枚奖牌位居第一；中国代表团以40金27银24铜91枚奖牌位列第二，40枚金牌的成绩超越2012年伦敦奥运会39枚金牌的纪录，创造了新的境外参 ......
  信息3: 完全奖牌榜. 排名; 国家/地区; 总数. 央视网AI网页生成技术支持....


=== Round 2 ===
Raw_responses:
<think>
好的，我现在需要处理用户的问题：“请告诉我2024年夏季奥运会的最新奖牌榜排名情况”。首先，我要检查提供的搜索结果是否足够回答这个问题。

用户的问题是关于奖牌榜的排名，所以我要看搜索结果中是否有相关的具体数据。信息1提到巴黎奥运会的奖牌榜，但没有具体数字。信息2和信息4提供了详细的数据，比如美国以5金13银12铜，中国以40金27银24铜，这应该是具体的排名。信息5提到了美国的奖牌数，但可能不够详细。信息3和4更详细，但可能重复了之前的资料。

现在，我需要确认是否有足够的信息来回答用户。信息2和信息4明确提到了美国和中国的奖牌数，以及具体排名。因此，这些信息应该足够回答用户的问题。接下来，我需要根据步骤一，直接回答用户，包括具体数字和排名，并列出参考来源。不需要使用

In [139]:
print("\n### Test 4: 上海天气 ###")
user_query = "上海2025年11月25日的天气怎么样?"
rounds4, sources4 = run_search_agent(user_query, max_rounds=5)
show_rounds(rounds4, sources4, user_query)


### Test 4: 上海天气 ###
Query: 上海2025年11月25日的天气怎么样?

=== Round 1 ===
Raw_responses:
<think>
好的，用户问的是上海2025年11月25日的天气怎么样。首先，我需要判断是否需要搜索。根据核心判断原则，如果需要实时数据或最新信息，比如天气，应该使用<search>标签。上海的天气通常会更新，所以可能需要实时数据。不过，用户的问题是具体的日期，可能已经有相关的天气信息。但如果没有，可能需要搜索。这时候应该生成一个简短的查询词，比如“上海2025年11月25日天气怎么样”。然后根据规则，直接回答，但需要确认是否有足够信息。如果有的话，列出参考来源，否则可能需要搜索。这里用户的问题明确是询问天气，所以可能不需要搜索，直接回答。
</think>

<search>上海2025年11月25日天气</search>

🔍 search_content: 上海2025年11月25日天气

📚 搜索到 5 条信息（仅展示前3条）:
  信息1: 上海市在十一月的每日氣溫變化幅度如何？ ... 月初的最高溫約為20°C，最低溫約為12.1°C。11日至20日期間，最高溫平均為17.3°C，最低溫平均為10.4°C。接近月底時，最高溫約為15.3...
  信息2: 五日天气预报. 上海中心气象台2025年11月24日10时发布. 11月24日星期一; 多云. 晴. 13℃～20℃; 西北风 风力4-5级. 11月25日星期二; 晴. 多云. 8℃～13℃; 西北...
  信息3: 溫度圖. °F. 80° 70° 60° 50° 40° 11月 ... 未來. 十二月2025 · 一月2026 · 二月2026. 全球範圍. 颶風 · 惡劣天氣 · 雷達與氣象圖 · 視訊....


=== Round 2 ===
Raw_responses:
<think>
好的，我现在需要处理用户的问题：“上海2025年11月25日的天气怎么样?”。首先，我要检查是否需要使用<search>标签来获取信息。

根据核心判断原则，如果问题需要实时数据、最新信息、具体事实或不确定答案，就需要搜索。这里用户的问题是关于天气的，所以需要查看提供的搜索结果。

查看搜索结果，信息1提到上海市在十一月2025的天气：气温变